In [4]:
# LP DAAC HLS API
# LP DAAC = Land Processes Distributed Active Archive Center
# HLS = Harmonized Landsat Sentinel-2

import requests as r

stac = "https://cmr.earthdata.nasa.gov/stac/"
stac_response = r.get(stac).json()
for s in stac_response: print(s)

id
title
stac_version
type
description
links


In [5]:
stac_lp = [s for s in stac_response['links'] if 'LP' in s['title']] # LP catalogs
for s in stac_lp: print(s)

{'title': 'LPDAAC_ECS', 'rel': 'child', 'type': 'application/json', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPDAAC_ECS'}
{'title': 'LPCUMULUS', 'rel': 'child', 'type': 'application/json', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCUMULUS'}
{'title': 'LPCLOUD', 'rel': 'child', 'type': 'application/json', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'}


In [6]:
lp_cloud = r.get([s for s in stac_lp if s['title'] == 'LPCLOUD'][0]['href']).json()
for l in lp_cloud: print(f"{l}: {lp_cloud[l]}")

# if rel['next'] exists we need to recursively call the next page
# if rel['next'] does not exist we are on the last page
def get_page(url, page=1):
    print (f"PAGE {page}")
    response = r.get(url).json()
    response_links = response['links']
    for l in response_links:
        print(f"{l['rel']}: {l['href']}")
    
    last_link = response_links[-1]
    print()

    if 'next' in last_link['rel']:
        get_page(last_link['href'], page + 1)
    else:
        print("Last page")

print()
print()
get_page(lp_cloud['links'][0]['href'])

id: LPCLOUD
title: LPCLOUD
description: Root catalog for LPCLOUD
type: Catalog
stac_version: 1.0.0
links: [{'rel': 'self', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD', 'title': 'Provider catalog', 'type': 'application/json'}, {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/', 'title': 'Root catalog', 'type': 'application/json'}, {'rel': 'collections', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections', 'title': 'Provider Collections', 'type': 'application/json'}, {'rel': 'search', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search', 'title': 'Provider Item Search', 'type': 'application/geo+json', 'method': 'GET'}, {'rel': 'search', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search', 'title': 'Provider Item Search', 'type': 'application/geo+json', 'method': 'POST'}, {'rel': 'conformance', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/conformance', 'title': 'Conformance Classes', 'type': 'application/geo+json'}, {'rel': 'ser

In [7]:
lp_links = lp_cloud['links']
for l in lp_links:
    try:
        print(f"{l['href']} is the {l['title']}.")
    except:
        print(f"{l['href']}")

https://cmr.earthdata.nasa.gov/stac/LPCLOUD is the Provider catalog.
https://cmr.earthdata.nasa.gov/stac/ is the Root catalog.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections is the Provider Collections.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search is the Provider Item Search.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search is the Provider Item Search.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/conformance is the Conformance Classes.
https://api.stacspec.org/v1.0.0-beta.1/openapi.yaml is the OpenAPI Doc.
https://api.stacspec.org/v1.0.0-beta.1/index.html is the HTML documentation.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/ASTGTM_NUMNC.v003
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/ASTGTM_NC.v003
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/ASTGTM.v003
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/WaterBalance_Daily_Historical_GRIDMET.v1.5
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/ECO_L2G_CLOUD.v002
https://

In [8]:
lp_collections = [l['href'] for l in lp_links if l['rel'] == 'collections'][0] # Set collections endpoint to variable
collections_response = r.get(f"{lp_collections}").json()
print(f"This collection contains {collections_response['description']} ({len(collections_response['collections'])} available).")
print("Here's our titles:")
for c in collections_response['collections']:
    print(c['title'])

This collection contains All collections provided by LPCLOUD (10 available).
Here's our titles:
ASTER Global Digital Elevation Model Attributes NetCDF V003
ASTER Global Digital Elevation Model NetCDF V003
ASTER Global Digital Elevation Model V003
Daily Historical Water Balance Products for the CONUS
ECOSTRESS Gridded Cloud Mask Instantaneous L2 Global 70 m V002
ECOSTRESS Gridded Land Surface Temperature and Emissivity Instantaneous L2 Global 70 m V002
ECOSTRESS Gridded Top of Atmosphere Calibrated Radiance Instantaneous L1C Global 70 m V002
ECOSTRESS Swath Attitude and Ephemeris Instantaneous L1B Global V002
ECOSTRESS Swath Cloud Mask Instantaneous L2 Global 70 m V002
ECOSTRESS Swath Geolocation Instantaneous L1B Global 70 m V002


Turns out NASA paginates their APIs. Incredibly annoying.

I'm just gonna skip to the good part and go to the actual HLS data - https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0
This was on page 3. That's why I was so confused trying to find it.

In [9]:
hls_data = r.get('https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0').json()
for s in hls_data: print(f"{s}: {hls_data[s]}")
for s in hls_data['links']: print(f"{s['href']} is the {s['title']}.")
hls_data['extent']

id: HLSL30.v2.0
stac_version: 1.0.0
license: not-provided
title: HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0
type: Collection
description: The Harmonized Landsat Sentinel-2 (HLS) project provides consistent surface reflectance (SR) and top of atmosphere (TOA) brightness data from a virtual constellation of satellite sensors. The Operational Land Imager (OLI) is housed aboard the joint NASA/USGS Landsat 8 and Landsat 9 satellites, while the Multi-Spectral Instrument (MSI) is mounted aboard Europe’s Copernicus Sentinel-2A and Sentinel-2B satellites. The combined measurement enables global observations of the land every 2–3 days at 30-meter (m) spatial resolution. The HLS project uses a set of algorithms to obtain seamless products from OLI and MSI that include atmospheric correction, cloud and cloud-shadow masking, spatial co-registration and common gridding, illumination and view angle normalization, and spectral bandpass adjustment.


{'spatial': {'bbox': [[-180, -90, 180, 90]]},
 'temporal': {'interval': [['2013-04-11T00:00:00.000Z', None]]}}

In [10]:
data2023 = [s for s in hls_data['links'] if '2023' in s['href']][0]['href']
print(data2023)

data2023_response = r.get(data2023).json()
for s in data2023_response: print(f"{s}: {data2023_response[s]}")
for s in data2023_response['links']: print(f"{s['href']} is the {s['title']}.")

https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023
id: HLSL30.v2.0-2023
title: HLSL30.v2.0 2023
description: LPCLOUD sub-catalog for 2023
links: [{'rel': 'root', 'title': 'Root Catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac', 'type': 'application/json'}, {'rel': 'self', 'title': 'HLSL30.v2.0 2023', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023', 'type': 'application/json'}, {'rel': 'parent', 'title': 'Parent Catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-01 catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/01', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-02 catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/02', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-03 catalog', 'href': 'https://cmr.earthdata.nasa.g

You see that ID? HLSL30.v2.0-2023? That's the ID we need to get the data.

In [11]:
data23_10 = [s for s in data2023_response['links'] if '2023/10' in s['href']][0]['href']
print(data23_10)

data23_10_response = r.get(data23_10).json()
for s in data23_10_response: print(f"{s}: {data23_10_response[s]}")
for s in data23_10_response['links']: print(f"{s['href']} is the {s['title']}.")

https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10
id: HLSL30.v2.0-2023-10
title: HLSL30.v2.0 2023-10
description: LPCLOUD sub-catalog for 2023-10
links: [{'rel': 'root', 'title': 'Root Catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac', 'type': 'application/json'}, {'rel': 'self', 'title': 'HLSL30.v2.0 2023-10', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10', 'type': 'application/json'}, {'rel': 'parent', 'title': 'Parent Catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-10-01 catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10/01', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-10-02 catalog', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10/02', 'type': 'application/json'}, {'rel': 'child', 'title': '2023-10-03 catalog',

In [17]:
data23_10_05 = [s for s in data23_10_response['links'] if '2023/10/05' in s['href']][0]['href']
print(data23_10_05)

data23_10_05_response = r.get(data23_10_05).json()
for s in data23_10_05_response: print(s)
for s in data23_10_05_response['links'][0:10]: print(f"{s['href']} is the {s['title']}.")
print("...")

https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10/05
id
title
description
links
type
stac_version
https://cmr.earthdata.nasa.gov/stac is the Root Catalog.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10/05 is the HLSL30.v2.0 2023-10-05.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/2023/10 is the Parent Catalog.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T56MNB.2023278T000010.v2.0 is the HLS.L30.T56MNB.2023278T000010.v2.0.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T56MNA.2023278T000010.v2.0 is the HLS.L30.T56MNA.2023278T000010.v2.0.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T56MMB.2023278T000010.v2.0 is the HLS.L30.T56MMB.2023278T000010.v2.0.
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T56MMA.2023278T000010.v2.0 is the HLS.L30.T56MMA.2023278T000010.v2.0.
ht

Now we've FINALLY gotten to individual grid designators. Let's take the first item we see and see what we can do with it.

In [38]:
rand_item = [s for s in data23_10_05_response['links'] if 'item' in s['rel']][54]['href']
print(rand_item)

rand_item_response = r.get(rand_item).json()

for s in rand_item_response: print(s)

print(f"""
ID: {rand_item_response['id']}
Date: {rand_item_response['properties']['datetime']}
Captured over: {rand_item_response['bbox']} (Lower Left, Upper Right)
Contains {len(rand_item_response['assets'])} assets
Cloud Cover: {rand_item_response['properties']['eo:cloud_cover']}%
      """)

print("Here are the assets:")
for a in rand_item_response['assets']: print(a)

https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T55JDL.2023278T000633.v2.0
type
id
stac_version
stac_extensions
collection
geometry
bbox
links
properties
assets

ID: HLS.L30.T55JDL.2023278T000633.v2.0
Date: 2023-10-05T00:06:33.161Z
Captured over: [146.178865, -27.210766, 147.098253, -26.218279] (Lower Left, Upper Right)
Contains 17 assets
Cloud Cover: 0%
      
['https://stac-extensions.github.io/eo/v1.0.0/schema.json']
Here are the assets:
B07
VAA
B03
B11
VZA
B01
B02
SAA
Fmask
B09
B06
B04
B05
B10
SZA
browse
metadata


Interesting note: The higher the number, the more cloud cover there was. Not sure if this is just sampling bias or if it's actually a thing.

In [35]:
print(rand_item_response['assets']['browse'])
print(rand_item_response['assets']['Fmask'])
print(rand_item_response['assets']['B01'])

{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSL30.020/HLS.L30.T56LKP.2023278T000209.v2.0/HLS.L30.T56LKP.2023278T000209.v2.0.jpg', 'type': 'image/jpeg', 'title': 'Download HLS.L30.T56LKP.2023278T000209.v2.0.jpg'}
{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T56LKP.2023278T000209.v2.0/HLS.L30.T56LKP.2023278T000209.v2.0.Fmask.tif', 'title': 'Download HLS.L30.T56LKP.2023278T000209.v2.0.Fmask.tif'}
{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T56LKP.2023278T000209.v2.0/HLS.L30.T56LKP.2023278T000209.v2.0.B01.tif', 'title': 'Download HLS.L30.T56LKP.2023278T000209.v2.0.B01.tif'}


So now we know how to get the data. However, we need to be able to query it by location (lat/lon). Luckily, if we back up a bit, one of the LPCLOUD links is a search API. Let's see what we can do with that.

In [46]:
geometry = {
    "type": "MultiPoint",
	"coordinates": [
		# Ames Research Center
		[
			-122.051522,
			37.406939
		],
		[
			-122.034953,
			37.426095
		]
	]
}

search_params = {
    "collections": ["HLSL30.v2.0"],
	"intersects": geometry,
	"datetime": "2023-10-05T00:00:00Z"
}

search_link = "https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search"
search_response = r.post(search_link, json=search_params).json()
for s in search_response['features'][0]: print(s)

type
id
stac_version
stac_extensions
collection
geometry
bbox
links
properties
assets


In [49]:
import geoviews as gv

base = gv.tile_sources.EsriImagery.opts(width=500,height=500)

geojson = {
    "type":"FeatureCollection",
    "features":
        [
            {
                "type":"Feature",
                "properties":{},
                "geometry": geometry
            }
        ]
}

area_of_interest = gv.Polygons([geometry]).opts(fill_alpha=0, line_color='red', line_width=2)

base * area_of_interest

DataError: None of the available storage backends were able to support the supplied data format. DictInterface raised following error:

 Dictionary data not understood, should contain a column per dimension or a mapping between key and value dimension values.

DictInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html